# 📊 Análise Exploratória - Score de Crédito QuantumFinance

## Objetivo
Explorar e entender o dataset de score de crédito para desenvolver um modelo de classificação.

## Autor
Aluno MBA FIAP - MLOps

---

## 1. Importar Bibliotecas

In [ ]:
# Bibliotecas para análise de dados
import pandas as pd
import numpy as np

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

# Configurar pandas para mostrar mais colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliotecas importadas com sucesso!")

## 2. Carregar Dados

In [ ]:
# Definir caminho dos dados
import os
from pathlib import Path

# Subir um nível do notebook para acessar a pasta data
DATA_PATH = Path('../data/raw')

# Listar arquivos disponíveis
print("📁 Arquivos disponíveis em data/raw:")
if DATA_PATH.exists():
    for file in DATA_PATH.glob('*.csv'):
        print(f"  - {file.name}")
else:
    print("❌ Pasta data/raw não encontrada!")

In [ ]:
# Carregar o dataset
# Ajustar o nome do arquivo conforme o que foi baixado do Kaggle
df = pd.read_csv(DATA_PATH / 'train.csv')  # ou outro nome do arquivo

print(f"✅ Dataset carregado com sucesso!")
print(f"📊 Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")

## 3. Visão Geral dos Dados

In [ ]:
# Primeiras linhas do dataset
print("📋 Primeiras 5 linhas do dataset:")
df.head()

In [ ]:
# Informações gerais
print("📊 Informações do Dataset:")
df.info()

In [ ]:
# Estatísticas descritivas das variáveis numéricas
print("📈 Estatísticas Descritivas:")
df.describe()

## 4. Análise da Variável Target (Credit Score)

In [ ]:
# Verificar a distribuição do Credit Score
print("🎯 Distribuição do Credit Score:")
print(df['Credit_Score'].value_counts())
print("\nPercentual:")
print(df['Credit_Score'].value_counts(normalize=True) * 100)

In [ ]:
# Visualizar distribuição do Credit Score
plt.figure(figsize=(8, 6))
df['Credit_Score'].value_counts().plot(kind='bar')
plt.title('Distribuição do Credit Score', fontsize=16, fontweight='bold')
plt.xlabel('Credit Score', fontsize=12)
plt.ylabel('Quantidade', fontsize=12)
plt.xticks(rotation=0)

# Adicionar valores nas barras
for i, v in enumerate(df['Credit_Score'].value_counts()):
    plt.text(i, v + 100, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Análise de Valores Faltantes

In [ ]:
# Verificar valores faltantes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Coluna': missing_values.index,
    'Valores_Faltantes': missing_values.values,
    'Percentual': missing_percent.values
})

# Mostrar apenas colunas com valores faltantes
missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Percentual', ascending=False)

if len(missing_df) > 0:
    print("❓ Colunas com valores faltantes:")
    print(missing_df)
    
    # Visualizar valores faltantes
    plt.figure(figsize=(10, 6))
    missing_df.plot(x='Coluna', y='Percentual', kind='bar')
    plt.title('Percentual de Valores Faltantes por Coluna', fontsize=16, fontweight='bold')
    plt.xlabel('Coluna', fontsize=12)
    plt.ylabel('Percentual (%)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("✅ Não há valores faltantes no dataset!")

## 6. Análise das Variáveis Numéricas

In [ ]:
# Identificar colunas numéricas
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"📊 Encontradas {len(numeric_cols)} colunas numéricas:")
print(numeric_cols)

In [ ]:
# Criar histogramas para variáveis numéricas
if len(numeric_cols) > 0:
    fig, axes = plt.subplots(nrows=(len(numeric_cols) + 3) // 4, ncols=4, figsize=(16, 4 * ((len(numeric_cols) + 3) // 4)))
    axes = axes.flatten()
    
    for i, col in enumerate(numeric_cols[:16]):  # Limitar a 16 gráficos
        df[col].hist(bins=30, ax=axes[i], edgecolor='black')
        axes[i].set_title(f'Distribuição: {col}', fontsize=10)
        axes[i].set_xlabel(col, fontsize=8)
        axes[i].set_ylabel('Frequência', fontsize=8)
    
    # Remover eixos extras
    for i in range(len(numeric_cols), len(axes)):
        fig.delaxes(axes[i])
    
    plt.suptitle('Distribuição das Variáveis Numéricas', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 7. Análise das Variáveis Categóricas

In [ ]:
# Identificar colunas categóricas
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remover Credit_Score da lista se estiver presente
if 'Credit_Score' in categorical_cols:
    categorical_cols.remove('Credit_Score')

print(f"📊 Encontradas {len(categorical_cols)} colunas categóricas:")
print(categorical_cols)

In [ ]:
# Analisar variáveis categóricas com poucos valores únicos
for col in categorical_cols[:10]:  # Limitar a 10 colunas
    unique_count = df[col].nunique()
    
    if unique_count <= 10:  # Mostrar apenas se tiver até 10 valores únicos
        print(f"\n📌 {col} ({unique_count} valores únicos):")
        print(df[col].value_counts())
        
        # Criar gráfico de barras
        plt.figure(figsize=(8, 4))
        df[col].value_counts().plot(kind='bar')
        plt.title(f'Distribuição: {col}', fontsize=14)
        plt.xlabel(col, fontsize=12)
        plt.ylabel('Quantidade', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

## 8. Análise de Correlação

In [ ]:
# Criar matriz de correlação apenas para variáveis numéricas
if len(numeric_cols) > 1:
    # Calcular correlação
    correlation_matrix = df[numeric_cols].corr()
    
    # Criar heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, 
                annot=True, 
                cmap='coolwarm', 
                center=0, 
                fmt='.2f',
                square=True,
                linewidths=1,
                cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação das Variáveis Numéricas', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 9. Análise Bivariada - Relação com Credit Score

In [ ]:
# Analisar algumas variáveis numéricas importantes em relação ao Credit Score
# Vamos pegar as primeiras 4 variáveis numéricas como exemplo
important_numeric_cols = numeric_cols[:4]

if len(important_numeric_cols) > 0:
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(important_numeric_cols):
        df.boxplot(column=col, by='Credit_Score', ax=axes[i])
        axes[i].set_title(f'{col} por Credit Score', fontsize=12)
        axes[i].set_xlabel('Credit Score', fontsize=10)
        axes[i].set_ylabel(col, fontsize=10)
    
    plt.suptitle('Análise Bivariada: Variáveis Numéricas vs Credit Score', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 10. Insights e Conclusões

In [ ]:
# Resumo dos insights encontrados
print("📋 RESUMO DA ANÁLISE EXPLORATÓRIA")
print("=" * 50)
print(f"\n1. Dataset:")
print(f"   - Total de registros: {df.shape[0]:,}")
print(f"   - Total de features: {df.shape[1]}")
print(f"   - Features numéricas: {len(numeric_cols)}")
print(f"   - Features categóricas: {len(categorical_cols)}")

print(f"\n2. Variável Target (Credit Score):")
for score, count in df['Credit_Score'].value_counts().items():
    percent = (count / len(df)) * 100
    print(f"   - {score}: {count:,} ({percent:.1f}%)")

print(f"\n3. Qualidade dos Dados:")
total_missing = df.isnull().sum().sum()
if total_missing > 0:
    print(f"   - Valores faltantes: {total_missing:,} ({(total_missing / (df.shape[0] * df.shape[1]) * 100):.2f}%)")
else:
    print(f"   - Sem valores faltantes ✅")

print(f"\n4. Próximos Passos:")
print("   - Tratar valores faltantes (se houver)")
print("   - Codificar variáveis categóricas")
print("   - Normalizar/padronizar features numéricas")
print("   - Selecionar features mais relevantes")
print("   - Treinar modelo de classificação")
print("   - Implementar validação cruzada")
print("   - Otimizar hiperparâmetros")
print("\n" + "=" * 50)

## 11. Salvar Dataset Limpo

In [ ]:
# Opção para salvar o dataset após análise inicial
# df.to_csv('../data/processed/credit_score_eda.csv', index=False)
# print("✅ Dataset salvo em data/processed/credit_score_eda.csv")